# Deepersense: sim data generator

## imports

In [1]:
import os
# import sys
import rospy
import rosbag
import rospkg
import glob

In [2]:
from simulation_controller import SimulationController
from scene_generator import SceneGenerator, ColorMap
from launchfile_maker import LaunchFileMaker

## file manager

### cwd

In [3]:
cwd = os.getcwd()

### package path

In [4]:
rospack = rospkg.RosPack()
package_path = rospack.get_path('stonefish_scene_generator')

### iterate files in folder

In [5]:
all_files_path = os.path.join(package_path,'*')
for file_name in glob.glob(all_files_path):
    print(file_name)

/home/zeged/catkin_ws/src/stonefish_scene_generator/launch
/home/zeged/catkin_ws/src/stonefish_scene_generator/src
/home/zeged/catkin_ws/src/stonefish_scene_generator/package.xml
/home/zeged/catkin_ws/src/stonefish_scene_generator/CMakeLists.txt


### create data folder

In [6]:
def create_folder(path):
    dir_exists = os.path.isdir(path)
    if not dir_exists:
        os.mkdir(path)
        print('data folder created: {}'.format(path))

In [7]:
data_folder_path = os.path.join(package_path, 'data')
create_folder(data_folder_path)

data folder created: /home/zeged/catkin_ws/src/stonefish_scene_generator/data


## generate stonefish scene

### generate scene

In [8]:
title = 'basic'
seabed_depth = 15
gen = SceneGenerator(seabed_depth)
gen.add_empty_frame(ColorMap.jet)
gen.add_reefs()
scene = gen.generate()
# print(scene)

### save scene to .scn file

In [9]:
scene_file_path = os.path.join(data_folder_path, title + '.scn')
f = open(scene_file_path, "w")
f.write(scene)
f.close()

## Simulation

### get launch_file_path

In [10]:
launch_file_name = 'auto_generated.launch'
launch_file_path = os.path.join(package_path, 'launch', launch_file_name)
print(launch_file_path)

/home/zeged/catkin_ws/src/stonefish_scene_generator/launch/auto_generated.launch


### edit/make launch file

In [11]:
rate = 300.0
resolution = (1200, 800)
output_bagfile_path = os.path.join(data_folder_path, title + '.bag')
maker = LaunchFileMaker(scene_file_path, rate, resolution, output_bagfile_path)
maker.save_to_file(launch_file_path)

### setup simulation

In [27]:
controller = SimulationController(launch_file_path)

... logging to /home/zeged/.ros/log/702b7748-28bb-11ec-ab87-107b448dffed/roslaunch-zeged-System-Product-Name-25998.log


### launch simulation

In [28]:
controller.run()
controller.wait_for_ros()

[INFO] [1633754607.555561]: started process
[INFO] [1633754607.570304]: waiting for topic msg
waiting for message
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
wait is over
rospy.get_time() - self.timer > 15
(1633754641.078895, 1633754626.078346)
[INFO] [1633754641.079056]: timeout: shutting down


## Bag

### open bag file

In [17]:
import rospy
from collections import defaultdict
import cv2
import numpy as np
from cv_bridge import CvBridge

bag = rosbag.Bag(bagfile, 'r')

## open bag file

In [18]:
bagfiles = glob.glob(os.path.join(data_folder_path, '*.bag'))
print(bagfiles)
bagfile_path = bagfiles[0]
bag = rosbag.Bag(bagfile_path, 'r')

['/home/zeged/catkin_ws/src/stonefish_scene_generator/data/basic_2021-10-09-07-42-22.bag']


## create a folder for images

In [19]:
image_folder_path = bagfile_path.split('.')[0]
print(image_folder_path)
create_folder(image_folder_path)

/home/zeged/catkin_ws/src/stonefish_scene_generator/data/basic_2021-10-09-07-42-22
data folder created: /home/zeged/catkin_ws/src/stonefish_scene_generator/data/basic_2021-10-09-07-42-22


In [20]:
def imshow(win, img):
    cv2.imshow(win, img)
    cv2.waitKey(10)
     

In [21]:
def process_image(msg, topic):
    bridge = CvBridge()
    encoding = 'bgr8'
    img = np.asarray(bridge.imgmsg_to_cv2(msg, encoding))
    return img

In [22]:
def process_depth(msg, topic):
    bridge = CvBridge()
    encoding = '32FC1'
    img = np.asarray(bridge.imgmsg_to_cv2(msg, encoding))
    return img

In [23]:
def normaliaze_depth(msg):
    cv_image_array = np.array(msg, dtype = np.dtype('f8'))
    slice1Copy = cv_image_norm = cv2.normalize(cv_image_array, cv_image_array, 0, 1, cv2.NORM_MINMAX)
    slice1Copy = np.uint8(slice1Copy*255)
    return(slice1Copy)

In [24]:
def save_image(img, filename):
    # print(file_name)
    cv2.imwrite(filename, img)

In [25]:
iterator = bag.read_messages()
counter = defaultdict(lambda: 0)
image_num = 0
for topic, msg, time in iterator:
    msg_time = rospy.Time.to_sec(time) # .to_time(time)
    msg_type = str(type(msg)).split('__')[1].split("'")[0]
    print(msg_time, msg_type, topic)
    counter[topic]+=1
    if msg_type == 'CameraInfo':
        continue
    image_num += 1

    if 'image_color' in topic or 'display' in topic:
        bridged_image = process_image(msg, str(topic))
        imshow(topic, bridged_image)
        file_name = os.path.join(image_folder_path, str(image_num) + '.tif')
        save_image(bridged_image, file_name)



    elif 'image_depth2/image_depth' in topic:
        bridged_image = process_depth(msg, str(topic))
        norm_image = normaliaze_depth(bridged_image)
        imshow(topic, norm_image)
        file_name = os.path.join(image_folder_path, str(image_num) + '.tif')
        save_image(norm_image, file_name)
    else:
        print(topic)

cv2.destroyAllWindows()

(1633754558.4905822, 'Image', '/sparus2/Blue_view_M900_FLS2/display')
(1633754558.5473194, 'CameraInfo', '/image_depth2/camera_info')
(1633754558.5475392, 'CameraInfo', '/camera2/image_raw/camera_info')
(1633754558.549733, 'Image', '/image_depth2/image_depth')
(1633754558.5514772, 'Image', '/camera2/image_raw/image_color')
(1633754558.5802357, 'Image', '/sparus2/Blue_view_M900_FLS2/display')
(1633754558.6352212, 'CameraInfo', '/image_depth2/camera_info')
(1633754558.6352563, 'CameraInfo', '/camera2/image_raw/camera_info')
(1633754558.6360917, 'Image', '/image_depth2/image_depth')
(1633754558.6369047, 'Image', '/camera2/image_raw/image_color')
(1633754558.7006793, 'CameraInfo', '/camera2/image_raw/camera_info')
(1633754558.7021232, 'Image', '/camera2/image_raw/image_color')
(1633754558.7029657, 'Image', '/sparus2/Blue_view_M900_FLS2/display')
(1633754558.7522964, 'CameraInfo', '/image_depth2/camera_info')
(1633754558.753297, 'CameraInfo', '/camera2/image_raw/camera_info')
(1633754558.75

In [26]:
print(counter)

defaultdict(<function <lambda> at 0x7fc6e7844d50>, {'/sparus2/Blue_view_M900_FLS2/display': 175, '/camera2/image_raw/image_color': 236, '/image_depth2/camera_info': 166, '/camera2/image_raw/camera_info': 236, '/image_depth2/image_depth': 166})
